In [33]:
import importlib
import utils.data_loader
import utils.tokenize
import models.embedding

#For reloading
importlib.reload(utils.data_loader)
importlib.reload(utils.tokenize)
importlib.reload(models.embedding)

<module 'models.embedding' from '/Users/bimalmgr/Downloads/deep_learning/transformers/trnsfmr_from_scratch/models/embedding.py'>

In [34]:
from utils.data_loader import DataLoader
from utils.tokenize import Tokenizer
from data import en,es
from models.embedding import Embeddings
from data import en,es
import torch
import torch.nn as nn

In [38]:
def reload_modules():
    %load_ext autoreload
    %autoreload 2

In [86]:
def main():
    reload_modules()
    print("This is the implementation of transformer from scratch in PyTorch.........!!")
    tokenizer = Tokenizer()
    data_loader = DataLoader(
        tokenize_source=tokenizer.tokenize_source,
        tokenize_target=tokenizer.tokenize_target
    )
    # train_set,valid_set,test_set = data_loader.make_dataset(es,en)
    # vocab_src,vocab_trg = data_loader.build_vocab(train_set,valid_set,test_set)
    # for word,index in vocab_trg.get_stoi().items():
    #     print(word,index)
    # print(f"Tokenized source example: {tokenizer.tokenize_source('Mi nombre es Bimal !! @ 1 23 * ;.')}")
    # print(f"Tokenized target example: {tokenizer.tokenize_target('My name is Bimal !! @ 1 23 * ;.')}")
    # print(f"Dataset examples: {data_loader.make_dataset(['Mi nombre es Bimal !! @ 1 23 * ;.'],['My name is Bimal !! @ 1 23 * ;.'])}")
    s_tr,s_v,s_te = data_loader.make_dataset(['Mi nombre ','es Bimal !!','@ 1 23 * ;.'],['My name','is Bimal !!','@ 1 23 * ;.'])
    print("train",s_tr,"val",s_v,"test",s_te)
    s_v,s_t = data_loader.build_vocab(s_tr,s_v,s_te)
    print(s_v.get_stoi())
    print(s_t.get_stoi())
    # print("Source vobularies:",vocab_src)
    embed = nn.Embedding(len(s_v),512)
    print(embed(torch.tensor([s_v.get_stoi()['Bimal'],s_v.get_stoi()['nombre']])))
    # print(embed(torch.tensor(vocab_src.get_stoi()['víctimas'])))
main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This is the implementation of transformer from scratch in PyTorch.........!!
Initializing the data loader........
train [{'src': ['<sos>', 'Mi', 'nombre', '<eos>'], 'trg': ['<sos>', 'My', 'name', '<eos>']}, {'src': ['<sos>', 'es', 'Bimal', '!', '!', '<eos>'], 'trg': ['<sos>', 'is', 'Bimal', '!', '!', '<eos>']}] val [] test [{'src': ['<sos>', '@', '1', '23', '*', ';', '.', '<eos>'], 'trg': ['<sos>', '@', '1', '23', '*', ';', '.', '<eos>']}]
Building Spanish Vocabulary.....
Done Building Spanish Vocabulary!
Buidling English Vocabulary.........
Done building English Vocabulary!
{'Bimal': 11, '23': 8, '1': 7, '.': 6, 'es': 13, '*': 5, '!': 4, ';': 9, '<sos>': 2, 'nombre': 14, 'Mi': 12, '@': 10, '<eos>': 3, '<pad>': 1, '<unk>': 0}
{'is': 13, 'Bimal': 11, '23': 8, '1': 7, 'name': 14, '.': 6, '*': 5, '!': 4, ';': 9, '<sos>': 2, '@': 10, '<eos>': 3, 'My': 12, '<pad>': 1, '<unk>': 0}
tensor([[-0.2161, -0.690